In [1]:
import xmlreader as xml
import re
import pandas as pd
import os
import gensim
import sys
import numpy as np
from utils import tokenize

DATA_PATH = '../database/'

In [2]:
train_docs = xml.readXML(DATA_PATH + "intertass2018-PE-train-tagged.xml")
test_docs  = xml.readXMLTest(DATA_PATH + "intertass2018-PE-test.xml" )

In [3]:
trn_text   = [t.content  for t in train_docs]
trn_label  = [t.polarity for t in train_docs]

test_text   = [t.content  for t in test_docs]
test_label  = [t.polarity for t in test_docs]

In [4]:
col_names = ['labels','text']

In [5]:
df_train = pd.DataFrame({'text':trn_text, 'labels':trn_label}, columns=col_names)
df_test  = pd.DataFrame({'text':test_text, 'labels':test_label}, columns=col_names)

In [6]:
CSV_PATH = DATA_PATH + 'csv_data/'

if not os.path.exists(CSV_PATH):
    os.makedirs(CSV_PATH)

In [7]:
df_train.to_csv(DATA_PATH + 'csv_data/train.csv', index=False)
df_test.to_csv(DATA_PATH  + 'csv_data/test.csv',  index=False)

In [8]:
df_train = pd.read_csv(DATA_PATH + 'csv_data/train.csv')
df_test = pd.read_csv(DATA_PATH + 'csv_data/test.csv')

In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
labels    1000 non-null int64
text      1000 non-null object
dtypes: int64(1), object(1)
memory usage: 15.7+ KB


In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1428 entries, 0 to 1427
Data columns (total 2 columns):
labels    1428 non-null object
text      1428 non-null object
dtypes: object(2)
memory usage: 22.4+ KB


In [11]:
tokenize(df_train['text'][0], None)

{'original': 'Sin ser fan de Juan Gabriel, siempre supe que era una fuerza de la naturaleza. Hoy escuché "Querida", y me dio una ternura enorme.',
 'words': ['sin',
  'ser',
  'fan',
  'de',
  'juan',
  'gabriel',
  'siempre',
  'supe',
  'que',
  'era',
  'una',
  'fuerza',
  'de',
  'la',
  'naturaleza',
  'hoy',
  'escuche',
  'querida',
  'y',
  'me',
  'dio',
  'una',
  'ternura',
  'enorme'],
 'ratio': 0.05,
 'clean': 'sin ser fan de juan gabriel, siempre supe que era una fuerza de la naturaleza. hoy escuche querida , y me dio una ternura enorme',
 'class': None}

In [12]:
len(df_train['text'])

1000

In [13]:
LOF_train = []
LOF_test  = []

for i in range(len(df_train['text'])):
    LOF_train.append(tokenize(df_train['text'][i], None)['words'])
    
for i in range(len(df_test['text'])):
    LOF_test.append(tokenize(df_test['text'][i], None)['words'])

In [14]:
print("Size of List Of Words (Train) : ", len(LOF_train))
print("Size of List Of Words (Test)  : ", len(LOF_test))

Size of List Of Words (Train) :  1000
Size of List Of Words (Test)  :  1428


In [15]:
LOF_test[0]

['siempre', 'hermosa', 'maria', 'gabriel']

In [16]:
from collections import Counter

cnt = Counter(word for doc in LOF_train + LOF_test for word in doc ).most_common()

In [17]:
len(cnt)

7861

In [19]:
cnt[:10]

[('que', 1341),
 ('de', 1198),
 ('y', 991),
 ('no', 777),
 ('la', 770),
 ('a', 769),
 ('me', 666),
 ('el', 662),
 ('en', 608),
 ('es', 506)]

In [20]:
def gensim_load_vec(path="SBW-vectors-300-min5.bin"):
    gensim_emb = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)
    vocab      = gensim_emb.index2word
    vec        = gensim_emb.syn0
    shape      = gensim_emb.syn0.shape
    return gensim_emb, vec, shape, vocab

In [21]:
gensim_emb, vec, shape, vocab = gensim_load_vec(DATA_PATH + "embeddings/SBW-vectors-300-min5.bin")

/home/jose/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  after removing the cwd from sys.path.
/home/jose/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """


In [22]:
'de' in vocab

True

In [23]:
vocab.index("de")

0

In [24]:
vocab.index("UNK")

148439

In [25]:
indexSeq_train = []
indexSeq_test  = []

unk_idx  = vocab.index("UNK")

In [26]:
for i, doc in enumerate(LOF_train):
    indexDoc = []
    for word in doc:
        if word in vocab:
            idx = vocab.index(word)
            indexDoc.append(idx)
        else:
            indexDoc.append(unk_idx)
     
    indexSeq_train.append(indexDoc)
    sys.stdout.write('\r{:5.2f}%'.format(100*(i+1)/len(LOF_train)))
sys.stdout.write('\rTrain Done!     \n\n') 

Train Done!     



In [27]:
for i, doc in enumerate(LOF_test):
    indexDoc = []
    for word in doc:
        if word in vocab:
            idx = vocab.index(word)
            indexDoc.append(idx)
        else:
            indexDoc.append(unk_idx)
            
    indexSeq_test.append(indexDoc)
    sys.stdout.write('\r{:5.2f}%'.format(100*(i+1)/(len(LOF_test))))
sys.stdout.write('\rTest Done!     \n\n') 

Test Done!     



In [28]:
TMP_PATH = PARENT_PATH + 'tmp/'

if not os.path.exists(TMP_PATH):
    os.makedirs(TMP_PATH)
    
np.save(DATA_PATH + 'tmp/train_labels.npy', trn_label)
np.save(DATA_PATH + 'tmp/test_labels.npy' , test_label)

NameError: name 'PARENT_PATH' is not defined

In [ ]:
trn_label = np.load(DATA_PATH + 'tmp/train_labels.npy')
test_label = np.load(DATA_PATH + 'tmp/test_labels.npy')

print(trn_label.size)
print(test_label.size)

In [ ]:
tok_train = np.array(indexSeq_train)
tok_test  = np.array(indexSeq_test)

np.save(DATA_PATH + 'tmp/tok_train.npy', tok_train)
np.save(DATA_PATH + 'tmp/tok_test.npy' , tok_test)

In [ ]:
tok_train = np.load(DATA_PATH + 'tmp/tok_train.npy')
tok_test  = np.load(DATA_PATH + 'tmp/tok_test.npy')

In [ ]:
for idx in tok_train[0]:
    print(vocab[idx], end =' ')

In [ ]:
df_train['text'][0]

In [ ]:
trn_label[0]

In [ ]:
for idx in tok_test[0]:
    print(vocab[idx], end =' ')